In [0]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Reshape

# UpSampling2D : 한 개의 값을 2X2의 4개 값으로 늘려 전체 그림을 키웁니다.
# Conv2DTranspose : Deconvolution 수행
from keras.layers import UpSampling2D, Conv2DTranspose

from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist

import numpy as np

# Google Colab에서 이미지를 보기 위한 cv2_imshow 함수
from google.colab.patches import cv2_imshow 

# Google Colab에서 출력창을 비우기 위한 clear_output 함수
from IPython.display import clear_output

In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 0 ~ 255 -> -1 ~ 1
x_train = (x_train.astype(np.float32) - 127.5)/127.5

print(np.shape(x_test))

# Channel 차원 추가
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)

print(np.shape(x_test))

11493376/11490434 [==============================] - 2s 0us/step
(10000, 28, 28)
(10000, 28, 28, 1)


In [0]:
# 생성기 모델 (Generator Model) 정의

generator_model = Sequential(name='Generator')

generator_model.add(Dense(input_dim=100, units=1024, activation = 'relu'))
generator_model.add(BatchNormalization())

generator_model.add(Dense(128*7*7, activation = 'relu'))
generator_model.add(BatchNormalization())

generator_model.add(Reshape((7, 7, 128), input_shape=(128*7*7,)))

generator_model.add(Conv2DTranspose(filters=128, kernel_size=(3,3), strides=2, activation='relu', padding='same'))
generator_model.add(Conv2D(64, (5, 5), padding='same', activation = 'relu'))
generator_model.add(BatchNormalization())

generator_model.add(Conv2DTranspose(filters=64, kernel_size=(3,3), strides=2, activation='relu', padding='same'))
generator_model.add(Conv2D(1, (5, 5), padding='same', activation = 'relu'))
generator_model.add(BatchNormalization())

generator_model.summary()












Model: "Generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              103424    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_2 (Dense)              (None, 6272)              6428800   
_________________________________________________________________
batch_normalization_2 (Batch (None, 6272)              25088     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 128)       147584    
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64

In [0]:
# 분별기 모델 (Discriminator Model) 정의

input_shape=(28,28,1)

discriminator_model = Sequential(name='Discriminator')

discriminator_model.add(Conv2D(64, (5, 5), padding='same', input_shape=input_shape, activation = 'relu'))
discriminator_model.add(MaxPooling2D(pool_size=(2, 2)))

discriminator_model.add(Conv2D(128, (5, 5), activation = 'relu'))
discriminator_model.add(MaxPooling2D(pool_size=(2, 2)))

discriminator_model.add(Flatten())

discriminator_model.add(Dense(1024, activation = 'sigmoid'))

discriminator_model.add(Dense(1))
discriminator_model.add(Activation('sigmoid'))

discriminator_model.summary()


Model: "Discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 128)       204928    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              3277824   
_________________________________________________________________
dense_4 (Dense)              (None, 1)              

In [0]:
# 생성 적대 신경망 모델(GAN Model)

gan_model = Sequential(name='GAN')

# generator_model 자체를 추가
gan_model.add(generator_model)

# trainable : discriminator_model의 가중치가 학습되지 않아야 할 때가 있으므로 설정해줍니다.
discriminator_model.trainable = False

# discriminator_model을 generator_model 뒷 층으로 추가
gan_model.add(discriminator_model)

gan_model.summary()

Model: "GAN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Generator (Sequential)       (None, 28, 28, 1)         6952645   
_________________________________________________________________
Discriminator (Sequential)   (None, 1)                 3485441   
Total params: 10,438,086
Trainable params: 6,937,923
Non-trainable params: 3,500,163
_________________________________________________________________


In [0]:
# 모델을 어떻게 학습 시킬지 정해줍니다.
generator_model.compile(loss='binary_crossentropy', optimizer="adam")
gan_model.compile(loss='binary_crossentropy', optimizer="adam")

# 이 설정을 통해 discriminator_model이 학습 가능할 때의 학습 방식을 정할 수 있습니다.
discriminator_model.trainable = True
discriminator_model.compile(loss='binary_crossentropy', optimizer="adam")



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# 학습 코드를 직접 짜줍니다

epochs = 10
batch_size = 32

# Epoch 만큼 반복합니다
for epoch in range(1,epochs):
  
  print("\nEpoch {}".format(epoch))
  
  # 전체 데이터셋 중 Mini-batch를 뽑아 학습해줍니다
  for index in range(int(x_train.shape[0]/batch_size)):
    
    # ---- 분별기 (Discriminator) 학습 ----

    # 생성기(Generator) 모델에 입력할 랜덤값을 생성합니다
    rand_vector = np.random.uniform(-1, 1, size=(batch_size, 100))
    
    # Mini-batch는 전체 데이터셋에서 순서대로 뽑습니다
    image_batch = x_train[index * batch_size : (index+1) * batch_size]
    
    # 랜덤값을 생성기(Generator)에 입력하여 출력된 이미지를 얻습니다 
    generated_images = generator_model.predict(rand_vector)
    
    # 데이터셋의 이미지와 생성기가 만든 이미지를 붙여줍니다
    X = np.concatenate((image_batch, generated_images))
    
    # X에 맞는 Label을 생성합니다. ex) y = [1, ... , 1, 0, ..., 0]
    y = [1] * batch_size + [0] * batch_size
    
    # train_on_batch : 한 Mini-batch에 대해 한 번만 학습을 실행하며, 목적 함수 값을 반환합니다
    #                  데이터 X, 라벨 y가 입력됩니다
    # d_loss : train_on_batch() 함수가 반환한 목적 함수 값을 갖습니다
    d_loss = discriminator_model.train_on_batch(X, y)
    

    # ---- 생성기 (Generator) 학습 ----

    # 전체 (GAN) 모델을 학습시키기 위해 랜덤값을 다시 생성해줍니다.
    rand_vector = np.random.uniform(-1, 1, (batch_size,  100))
    
    # 분별기 (Discriminator) 모델이 학습되지 않도록 고정시킵니다
    discriminator_model.trainable = False
    
    # 전체 (GAN) 모델 중 분별기 (Discriminator)를 고정한 모델을 학습시킵니다.
    g_loss = gan_model.train_on_batch(rand_vector, [1] * batch_size)
    
    # 다시 분별기 (Discriminator) 모델이 학습되도록 합니다.
    discriminator_model.trainable = True
    
    print("\rbatch {} d loss: {} g loss: {}".format(index, d_loss, g_loss), end="")
  
  clear_output()
  # ---- Epoch 마다 학습이 잘 되었는지 확인하기 위해 생성된 그림을 출력해봅시다 ----
  for i in range(10):
    # 생성기(Generator) 입력을 위한 랜덤값 생성
    rand_vector = np.random.uniform(-1, 1, size=(batch_size, 100))

    generated_images = generator_model.predict(rand_vector) # 그림 생성

    # 그림의 범위를 원래대로 돌려줍니다. -1 ~ 1 -> 0 ~ 255
    generated_images = (np.array(generated_images) * 127.5) + 127.5

    # ---- 그림을 한번 가로로 출력해봅시다 ----

    np_images = np.zeros((28,28)) # 빈 그림 생성
    np_images = generated_images[0,:,:,0] # 첫 그림 넣기

    # 나머지 그림 옆으로 이어붙히기
    for i in range(31):
      np_images = np.concatenate((np_images, generated_images[i+1,:,:,0]), axis=1)
  
    # 이미지 출력
    cv2_imshow(np_images)

In [0]:
# 학습된 GAN 모델로 무작위 손글씨 숫자를 생성해봅시다.
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow 

# 총 32 * 32 개의 무작위 손글씨 숫자 생성
for i in range(32):
  # 100 * 32의 -1 ~ 1 범위의 랜덤 벡터 생성
  rand_vector = np.random.uniform(-1, 1, size=(batch_size, 100))

  # 생성기에 랜덤 벡터 입력으로 무작위 숫자 생성 
  generated_images = generator_model.predict(rand_vector)

  # 픽셀을 시각화하기 위해 -1 ~ 1 -> 0 ~ 255
  generated_images = (np.array(generated_images) * 127.5) + 127.5

  # 첫 장 채워넣기
  np_images = np.zeros((28,28))
  np_images = generated_images[0,:,:,0]

  # 나머지 31장 옆으로 이어붙이기
  for i in range(31):
    np_images = np.concatenate((np_images, generated_images[i+1,:,:,0]), axis=1)
  
  # 이미지 출력
  cv2_imshow(np_images)


In [0]:
# 본인의 구글 드라이브 → 지금 실행중인 코드

# google.colab.drive : 구글 드라이브에서 파일을 가져오기 위한 코드를 담고 있다.
from google.colab import drive

# 본인의 구글 드라이브를 '/gdrive' 라는 경로로 하여 쓸 수 있다.
drive.mount('/gdrive', force_remount=True)

In [0]:
# 모델은 h5 파일 형태로 저장됩니다
autoencoder.save('/gdrive/My Drive/MNIST_GAN.h5')

# 모델을 불러오기 위해 지워줍니다
del autoencoder

# 저장되어 있는 모델을 불러오기 위한 load_model 함수
from keras.models import load_model

# 'model' 에 해당 모델을 불러옵니다
model = load_model('/gdrive/My Drive/MNIST_GAN.h5')